# Getting started with Xanthus

## What is Xanthus?

Xanthus is a Neural Recommender package written in Python. It started life as a personal project to take an academic ML paper and translate it into a 'production-ready' software package and to replicate the results of the paper along the way. It uses Tensorflow 2.0 under the hood, and makes extensive use of the Keras API. If you're interested, the original authors of [the paper that inspired this project]() provided code for their experiments, and this proved valuable when starting this project. However, while it is great that they provided their code, the repository isn't maintained, the code uses an old versions of Keras (and Theano!), it can be a little hard for beginners to get to grips with, and it's very much tailored to produce the results in their paper. All fair enough, they wrote a great paper and published their workings. Admirable stuff. Xanthus aims to make it super easy to get started with their work, and to scale their techniques (hopefully) gracefully with you as the complexity of your applications increase.

## Loading a sample dataset

Ah, the beginning of a brand new ML problem. Time to crack out Pandas and load some CSVs. You know the drill

In [1]:
import pandas as pd

ratings = pd.read_csv("../data/movielens-100k/ratings.csv")
movies = pd.read_csv("../data/movielens-100k/movies.csv")

Let's take a look at the data we've loaded. Here's the movies dataset:

In [2]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


As you can see, you've got the unique identifier for your movies, the title of the movie in human-readable format, and then the column `genres` that has a string containing a set of associated genres for the given movie. Straightforward enough. And hey, that `genres` column might come in handy at some point...

On to the `ratings` frame. Here's what is in there:

In [3]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


First up, you've got a `userId` corresponding to the unique user identifier, and you've got the `movieId` corresponding to the unique movie identifier (this maps onto the `movieId` column in the `movies` frame, above). You've also got a `rating` field. This is associated with the user-assigned rating for that movie. Finally, you have the `timestamp` -- the date at which the user rated the movie. For future reference, you can convert from this timestamp to a 'human readable' date with:

In [4]:
from datetime import datetime

datetime.fromtimestamp(ratings.iloc[0]["timestamp"]).strftime("%Y-%m-%d %H:%M:%S")

'2000-07-30 19:45:03'

Thats your freebie for the day. Onto getting the data ready for training your recommender model.

## Data preparation

Xanthus provides a few utilities for getting your recommender up and running. One of the more ubiquitous utilities is the `Dataset` class, and its related `DatasetEncoder` class. At the time of writing, the `Dataset` class assumes your 'ratings' data is in the format `user`, `item`, `rating`. You can rename the sample data to be in this format with:

In [5]:
ratings = ratings.rename(columns={"userId": "user", "movieId": "item"})

Next, you might find it helpful to re-map the movie IDs (now under the `item` column) to be the `titles` in the `movies` frame. This'll make it easier for you to see what the recommender is recommending! Don't do this for big datasets though -- it can get very expensive very quickly! Anyway, remap the `item` column with:

In [6]:
title_mapping = dict(zip(movies["movieId"], movies["title"]))
ratings.loc[:, "item"] = ratings["item"].apply(lambda _: title_mapping[_])
ratings.head(2)

,user,item,rating,timestamp
0,1,Toy Story (1995),4.0,964982703
1,1,Grumpier Old Men (1995),4.0,964981247


A little more meaningful, eh? For this example, you are going to be looking at _implicit_ recommendations, so should also remove clearly negative rating pairs from the dataset. You can do this with:

In [7]:
ratings = ratings[ratings["rating"] > 3.0]

### Leave one out protocol

As with any ML model, it is important to keep a held-out sample of your dataset to evaluate your model's performance. This is naturally important for recommenders too. However, recommenders differ slightly in that we are often interested in the recommender's ability to _rank_ candidate items in order to surface the most relevant content to a user. Ultimately, the essence of recommendation problems is search, and getting relevant items in the top `n` search results is generally the name of the game -- absolute accuracy can often be a secondary consideration.

One common way of evaluating the performance of a recommender model is therefore to create a test set by sampling `n` items from each user's `m` interactions (e.g. movie ratings), keeping `m-n` interactions in the training set and putting the 'left out' `n` samples in the test set. The thought process then goes that when evaluating a model on this test set, you should see the model rank the 'held' out samples more highly in the results (i.e. it has started to learn a user's preferences). 

The 'leave one out' protocol is a specific case of this approach where `n=1`. Concretely, when creating a test set using 'leave one out', you withold a single interaction from each user and put these in your test set. You then place all other interactions in your training set. To get you going, Xanthus provides a utility function called -- funnily enough -- `leave_one_out` under the `evaluate` subpackage. You can import it and use it as follows:

In [8]:
from xanthus.evaluate import leave_one_out

train_df, test_df = leave_one_out(ratings, shuffle=True, deduplicate=True)

You'll notice that there's a couple of things going on here. Firstly, the function returns the input interactions frame (in this case `ratings`) and splits it into the two datasets as expected. Fair enough. We then have two keyword arguments `shuffle` and `deduplicate`. The argument `shuffle` will -- you guessed it -- shuffle your dataset before sampling interactions for your test set. This is set to `True` by default, so it is shown here for the purpose of being explicit. The second argument is `deduplicate`. This does what you might expect too -- it strips any cases where a user interacts with a specific item more than once (i.e. a given user-item pair appears more than once).

As discussed above, the `leave_one_out` function is really a specific version of a more general 'leave `n` out' approach to splitting a dataset. There's also other ways you might want to split datasets for recommendation problems. For many of those circumstances, Xanthus provides a more generic `split` function. This was inspired by Azure's [_Recommender Split_](https://docs.microsoft.com/en-us/azure/machine-learning/studio-module-reference/split-data-using-recommender-split#:~:text=The%20Recommender%20Split%20option%20is,user%2Ditem%2Drating%20triples) method in Azure ML Studio. There are a few important tweaks in the Xanthus implementation, so make sure to check out that functions documentation if you're interested.

Anyway, time to build some datasets.

## Introducing the `Dataset`

Like other ML problems, recommendation problems typically need to create encoded representations of a domain in order to be passed into a model for training and evaluation. However, there's a few aspects of recommendation problems that can make this problem particularly fiddly. To help you on your way, Xanthus provides a few utilities, including the `Dataset` class and the `DatasetEncoder` class. These structures are designed to take care of the fiddliness for you. They'll build your input vectors (including with metadata, if you provide it -- more on that later) and sparse matrices as required. You shouldn't need to touch a thing. 

Here's how it works. First, your 'train' and 'test' datasets are going to need to share the same encodings, right? Otherwise they'll disagree on whether `Batman Forever (1995)` shares the same encoding across the datasets, and that would be a terrible shame. To create your `DatasetEncoder` you can do this:

In [9]:
from xanthus.datasets import DatasetEncoder

encoder = DatasetEncoder()
encoder.fit(ratings["user"], ratings["item"])

This encoder will store all of the unique encodings of every user and item in the `ratings` set. Notice that you're passing in the `ratings` set here, as opposed to either train or test. This makes doubly sure you're creating encodings for every user-item pair in the dataset. To check this has worked, you can call the `transform` method on the encoder like this:

In [10]:
encoder.transform(items=["Batman Forever (1995)"])

{'items': array([1694], dtype=int32)}

The naming conventions on the `DatasetEncoder` are deliberately reminicent of the methods on Scikit-Learn encoders, just to help you along with using them. Now you've got your encoder, you can create your `Dataset` objects:

In [11]:
from xanthus.datasets import Dataset, utils

train_ds = Dataset.from_df(train_df, normalize=utils.as_implicit, encoder=encoder)
test_ds = Dataset.from_df(test_df, normalize=utils.as_implicit, encoder=encoder)

Let's unpack what's going on here. The `Dataset` class provides the `from_df` class method for quickly constructing a `Dataset` from a 'raw' Pandas `DataFrame`. You want to create a train and test dataset, hence creating two separate `Dataset` objects using this method. Next, you can see that the `encoder` keyword argument is passed in to the `from_df` method. This ensures that each `Dataset` maintains a reference to the _same_ `DatasetEncoder` to ensure consistency when used. The final argument here is `normalize`. This expects a callable object (e.g. a function) that scales the `rating` column (if provided). In the case of this example, the normalization is simply to treat the ratings as an implicit recommendation problem (i.e. all zero or one). The `utils.as_implicit` function simply sets all ratings to one. Simple enough, eh?

And that is it for preparing your datasets for modelling, at least for now. Time for some Neural Networks.

## Getting neural

With your datasets ready, you can build and fit your model. In the example, the `GeneralizedMatrixFactorizationModel` (or `GMFModel`) is used. If you're not sure what a GMF model is, be sure to check out the original paper, and the GMF class itself in the Xanthus docs. Anyway, here's how you set it up: 

In [12]:
from xanthus.models import GeneralizedMatrixFactorizationModel as GMFModel

fit_params = dict(epochs=10, batch_size=256)

model = GMFModel(
    fit_params=fit_params, n_factors=32, negative_samples=4
)

What is going on here, you ask? Good question. First, you import the `GeneralizedMatrixFactorizationModel` as any other object. You then define `fit_params` -- fit parameters -- to define the training loop used by the Keras optimizer. All Xanthus neural recommender models inherit from the base `NeuralRecommenderModel` class. By default, this class (and therefore all child classes) utilize the `Adam` optimizer. You can configure this to use any optimizer you wish though!

After the `fit_param`, the `GeneralizedMatrixFactorizationModel` is initialized. There are two further keyword arguments here, `n_factors` and `negative_samples`. In the former case, `n_factors` refers to the size of the latent factor space encoded by the model. The larger the number, the more expressive the model -- to a point. In the latter case, `negative_samples` configures the sampling pointwise sampling policy outlined by [He et al](). In practice, the model will be trained by sampling 'negative' instances for each positive instance in the set. In other words: for each user-item pair with a positive rating (in this case one -- remember `utils.as_implicit`?), a given number of `negative_samples` will be drawn that the user _did not_ interact with. This is resampled in each epoch. This helps the model learn more general patterns, and to avoid overfitting. Empirically, it makes quite a difference over other sampling approaches. If you're interested, you should look at the [pairwise loss used in Bayesian Personalized Ranking (BPR)]().

You're now ready to fit your model. You can do this with:

In [13]:
model.fit(train_ds)

1075/1075 [==============================] - 2s 2ms/step - loss: 0.4906 - val_loss: 0.3608
Epoch 2/2
1075/1075 [==============================] - 2s 2ms/step - loss: 0.3338 - val_loss: 0.3290
Epoch 3/3
1075/1075 [==============================] - 2s 2ms/step - loss: 0.3080 - val_loss: 0.3011
Epoch 4/4
1075/1075 [==============================] - 2s 2ms/step - loss: 0.2869 - val_loss: 0.2776
Epoch 5/5
1075/1075 [==============================] - 2s 2ms/step - loss: 0.2674 - val_loss: 0.2717
Epoch 6/6
1075/1075 [==============================] - 2s 2ms/step - loss: 0.2483 - val_loss: 0.2476
Epoch 7/7
1075/1075 [==============================] - 2s 2ms/step - loss: 0.2307 - val_loss: 0.2295
Epoch 8/8
1075/1075 [==============================] - 2s 2ms/step - loss: 0.2125 - val_loss: 0.2110
Epoch 9/9
1075/1075 [==============================] - 2s 2ms/step - loss: 0.1955 - val_loss: 0.1968
Epoch 10/10
1075/1075 [==============================] - 2s 2ms/step - loss: 0.1795 - val_loss: 0.178

GeneralizedMatrixFactorizationModel()

## Evaluating the model

In [14]:
from xanthus.evaluate import he_sampling

_, test_items, _ = test_ds.to_components(shuffle=False)
users, items = he_sampling(test_ds, train_ds, n_samples=200)

In [15]:
recommended = model.predict(test_ds, users=users, items=items, n=10)

In [19]:
from xanthus.evaluate import score, metrics

print("t-nDCG", score(metrics.truncated_ndcg, test_items, recommended).mean())
print("HR@k", score(metrics.precision_at_k, test_items, recommended).mean())

t-nDCG 0.450049862096657
HR@k 0.694078947368421


## The fun bit

After all of that, it is time to see what you've won.

In [24]:
recommended = model.predict(users=users, items=items[:, 1:], n=5)

In [26]:
recommended_df = encoder.to_df(users, recommended)
recommended_df.head(25)

,id,item_0,item_1,item_2,item_3,item_4
0,1,"Godfather, The (1972)","Thomas Crown Affair, The (1999)",Boogie Nights (1997),Weird Science (1985),Wayne's World 2 (1993)
1,2,Harry Potter and the Deathly Hallows: Part 2 (...,Big Hero 6 (2014),Raiders of the Lost Ark (Indiana Jones and the...,"Bourne Identity, The (2002)",Ace Ventura: Pet Detective (1994)
2,3,"Paper, The (1994)",Rumble in the Bronx (Hont faan kui) (1995),"Arrival, The (1996)",Deep Impact (1998),Dolores Claiborne (1995)
3,4,Dances with Wolves (1990),Hoop Dreams (1994),Star Wars: Episode VI - Return of the Jedi (1983),Terminator 2: Judgment Day (1991),Boys Don't Cry (1999)
4,5,While You Were Sleeping (1995),"Net, The (1995)",Contact (1997),"Jungle Book, The (1994)",My Fair Lady (1964)
5,6,Wes Craven's New Nightmare (Nightmare on Elm S...,Cold Comfort Farm (1995),Star Wars: Episode IV - A New Hope (1977),American Pie (1999),"Colonel Chabert, Le (1994)"
6,7,Monty Python and the Holy Grail (1975),"Truman Show, The (1998)",Babel (2006),American Gangster (2007),Taken (2008)
7,8,"Matrix, The (1999)",Back to the Future (1985),Willy Wonka & the Chocolate Factory (1971),Die Hard (1988),2001: A Space Odyssey (1968)
8,9,Casablanca (1942),Rear Window (1954),It's a Wonderful Life (1946),Toy Story 2 (1999),Butch Cassidy and the Sundance Kid (1969)
9,10,"Chronicles of Narnia: Prince Caspian, The (2008)","Silence of the Lambs, The (1991)",Moana (2016),Cars (2006),Ace Ventura: When Nature Calls (1995)
